## HW2 part1, Jordan Gittleman

Hey Hanu! I'm still working on this because im slow and also stuborn! Come back later mabye if you're reading this =D

In [1]:
import os
import cv2

import matplotlib
import numpy as np

import matplotlib.pyplot as plt
%matplotlib tk

First, the pointLoad function loads in the specified image and prompts to hand pick a set amount of points. TODO: store points for future runs to disable popup's and embed images for ease of documentation

In [2]:
def pointLoad(imgpath,pts = 0, imgpass = True):

    if os.path.exists(imgpath):
        img = cv2.imread(imgpath)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if pts:
            ptMatrix = pointPick(img,pts)
            # plt.close()
            if imgpass:
                return img, np.array(ptMatrix).astype(int)
            else:
                return np.array(ptMatrix).astype(int)    
        else:
            return img
    else:
        print("Invalid image path")
        return
    
            

In [3]:
def pointPick(img,pts):
    plt.imshow(img)
    ptMatrix = plt.ginput(pts)
    return ptMatrix

getH creates a homography matrix between the source and destination photos, instead of using svd, I opted to construct the matrix with an explicit final h33 param = 1 and use least squares, something svd does internally anyway but I want to be difficult...

In [4]:
def getH(dest, src):
    p = np.array([0,0,0,0,0,0,0,0,1])
    # if len(dest) < 5 and len(src) < 5:
        

    for i in range(len(dest)):
        pi = np.array([[-src[i][0],-src[i][1],-1,0,0,0,src[i][0]*dest[i][0],src[i][1]*dest[i][0],dest[i][0]],
        [0,0,0,-src[i][0],-src[i][1],-1,src[i][0]*dest[i][1],src[i][1]*dest[i][1],dest[i][1]]])
        p = np.vstack((pi,p))

  
    x = np.zeros((len(dest)*2,1))
    x = np.vstack((x,np.array([1])))
    H, residuals, rank, s = np.linalg.lstsq(p,x)
    H = H.reshape((3,3))
    

    
    return H
        


function to paste the source image onto the destination, the implementation itself is a little hack-y

In [ ]:
def pasteImages(img1,img1pts,img2,img2pts):
    H = getH(img1pts,img2pts)
    im_out = cv2.warpPerspective(img2,H,(img1.shape[1],img1.shape[0]))
    mask = np.zeros(img1.shape[:2], dtype="uint8")
    cv2.fillConvexPoly(mask, np.int32([img1pts]),color=255)
    cv2.fillConvexPoly(img1, np.int32([img1pts]),color=0)
    masked =cv2.bitwise_and(im_out,im_out,mask=mask)
    img3 = cv2.add(img1,masked)
    # img3=cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
    return img3, H


extract 4 points from the images

In [ ]:
img1, img1pts = pointLoad("image1.jpg",4)

img2, img2pts = pointLoad("image2.jpg",4)



paste the images together

In [ ]:

img3, H3 = pasteImages(img1,img1pts,img2,img2pts)

and display...

In [ ]:

plt.imshow(img3)

plt.show()

This process can also be repeated with as many points as youd like due to the getH function performing least squares on the images, here we will do just that with 8 points from each image...

In [ ]:
img1New, img1ptsNew = pointLoad("image1.jpg",8)
img2New, img2ptsNew = pointLoad("image2.jpg",8)


In [ ]:
img3New, H3New = pasteImages(img1New,img1ptsNew,img2New,img2ptsNew)

In [ ]:
plt.imshow(img3New)

plt.show()

I can compare the two H matrix's and see how they differ...

In [ ]:
print(H3)
print("~~~~")
print(H3New)

Clearly, both Homographies differ while still yielding similar transoformations. In part, this is due to my bad clicking accuracy between images. A more major factor however, is that the transofrms are scaled differently.
If you observe the value at h33, the scale factors differ wildly, given that that h33 = wk where w is an arbitrary scale factor and k is 1. This difference for us does not matter fortunately as these transformations are scale invariant.

Now lets perform this 8 point homography again but for a new image!

In [ ]:
img4, img4pts = pointLoad("image3.jpg",8)

In [ ]:
img5, H5 = pasteImages(img1New,img1ptsNew,img4,img4pts)

plt.imshow(img5)


Now lets try some panorama stitching. This will work in much the same way as before but with some extra complications for our final output image.

In [ ]:


left={'pts':[],'h':[]}
right={'pts':[],'h':[]}
# get points/homographies for left-bound images
for n in [2,1]:
    destpts = pointLoad(image_paths[n],4,False)
    left['pts'].append(destpts)
    srcpts = pointLoad(image_paths[n-1],4,False)
    left['pts'].append(srcpts)
    left['h'].append(getH(destpts,srcpts))

for n in [2,3]:
    destpts = pointLoad(image_paths[n],4,False)
    right['pts'].append(destpts)
    srcpts = pointLoad(image_paths[n+1],4,False)
    right['pts'].append(srcpts)
    right['h'].append(getH(destpts,srcpts))
    



Now we have an array of the left homographies and of the right as well as points.

Creating this panorama is pretty tricky business, since as the panorama is created, the prev found points need to be shifted to keep up with the composite. There are a few appraoches to it, and given how numpy indexes things, its way easier to stitch to the right than to the left. Under lazy circumstances going just from left to right is fine but creating homographies that relate to a central image more directly (via matmul'ing with anotehr homography) will give a result with less drift.

In [18]:
# function found on stack overflow, I am trying to go through and understand how it works to be able to expand on it for a panorama/make my own approach, to do this, I'll add copmments explaining (mostly to myself) what's happening
def warpTwoImages(img1, img2, H):
    '''warp img2 to img1 with homograph H'''
    h1,w1 = img1.shape[:2] #grab height and width from each input image
    h2,w2 = img2.shape[:2]
    pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2) #
    pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    pts2_ = cv2.perspectiveTransform(pts2, H)
    pts = np.concatenate((pts1, pts2_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate

    result = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))
    result[t[1]:h1+t[1],t[0]:w1+t[0]] = img1
    return result


In [19]:

# image_paths=['i1.jpg','i2.jpg','i3.jpg','i4.jpg','i5.jpg']
# images=[]
# for image in image_paths:
#     img = pointLoad(image)
#     cv2.resize(img, (0,0), fx=.1, fy=.1)
#     images.append(img)
# trainImg=images[2]
# queryImg=images[1]
# width = trainImg.shape[1] + queryImg.shape[1]
# height = trainImg.shape[0] #+ queryImg.shape[0]

# result = cv2.warpPerspective(trainImg, getH(pointPick(trainImg,4),pointPick(queryImg,4)), (width, height))
# result[0:queryImg.shape[0], 0:queryImg.shape[1]] = queryImg

# plt.figure(figsize=(20,10))
# plt.imshow(result)

# plt.axis('off')
# plt.show()

# this is a super hack-y apprach, essentially, I am finding overlap points iterativly with each composite
# super unsatisying apprach so this is still a WiP I want to approach properly
output1 =warpTwoImages(images[2],images[1],getH(pointPick(images[2],4),pointPick(images[1],4)))
output2 =warpTwoImages(output1,images[0],getH(pointPick(output1,4),pointPick(images[0],4)))
# output3 =warpTwoImages(output2,images[1],getH(pointPick(output2,4),pointPick(images[1],4)))
# output4 =warpTwoImages(output3,images[0],getH(pointPick(output3,4),pointPick(images[0],4)))
# output2 =warpTwoImages(output1,images[3],(right['h'][0]))


/tmp/ipykernel_35992/3460832648.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  H, residuals, rank, s = np.linalg.lstsq(p,x)
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/jordan/anaconda3/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/home/jordan/anaconda3/lib/python3.9/tkinter/__init__.py", line 814, in callit
    func(*args)
  File "/home/jordan/anaconda3/lib/python3.9/site-packages/matplotlib/backends/_backend_tk.py", line 476, in delayed_destroy
    self.window.destroy()
  File "/home/jordan/anaconda3/lib/python3.9/tkinter/__init__.py", line 2311, in destroy
    for c in list(self.children.values()): c.destroy()
  File "/home/jordan/anaconda3/lib/python3.9/tkinte

TclError: 

ERROR:tornado.application:Exception in callback functools.partial(<function Kernel.enter_eventloop.<locals>.advance_eventloop at 0x7fa42ad6c940>)
Traceback (most recent call last):
  File "/home/jordan/anaconda3/lib/python3.9/site-packages/matplotlib/blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "/home/jordan/anaconda3/lib/python3.9/site-packages/matplotlib/backends/_backend_tk.py", line 385, in start_event_loop
    self._master.mainloop()
  File "/home/jordan/anaconda3/lib/python3.9/tkinter/__init__.py", line 1429, in mainloop
    self.tk.mainloop(n)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jordan/anaconda3/lib/python3.9/site-packages/matplotlib/backends/_backend_tk.py", line 113, in blit
    photoimage.tk.call(_blit_tcl_name, argsid)
_tkinter.TclError: invalid command name "pyimage173"

During handling of the above exception, another 

In [ ]:
plt.imshow(output4)

In [ ]:
plt.imshow(output1)

In [ ]:
plt.imshow(output2)